In [11]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


import os
import time
import numpy as np
import cv2
from skimage import feature, io, color
from sklearn.metrics import classification_report
import joblib
import json
import itertools
from tqdm import tqdm

from utils import *

In [12]:
# test_dir = "../data/test"
# test_data, test_labels = load_raw_data(test_dir)

train_dir = "../data/train"
train_data, train_labels = load_raw_data(train_dir)
train_labels = np.array(train_labels)

bins = [8, 16, 32, 64, 128, 256]
sizes = [32, 64, 128, 250, 512]  # default image_size = 250x250 

bins_h = [8, 16, 32, 64, 128, 180]
bins_s = [8, 16, 32, 64, 128, 256]
bins_v = [8, 16, 32, 64, 128, 256]

bin_combinations = list(itertools.product(bins_h, bins_s, bins_v))

In [13]:
print(bin_combinations)

[(8, 8, 8), (8, 8, 16), (8, 8, 32), (8, 8, 64), (8, 8, 128), (8, 8, 256), (8, 16, 8), (8, 16, 16), (8, 16, 32), (8, 16, 64), (8, 16, 128), (8, 16, 256), (8, 32, 8), (8, 32, 16), (8, 32, 32), (8, 32, 64), (8, 32, 128), (8, 32, 256), (8, 64, 8), (8, 64, 16), (8, 64, 32), (8, 64, 64), (8, 64, 128), (8, 64, 256), (8, 128, 8), (8, 128, 16), (8, 128, 32), (8, 128, 64), (8, 128, 128), (8, 128, 256), (8, 256, 8), (8, 256, 16), (8, 256, 32), (8, 256, 64), (8, 256, 128), (8, 256, 256), (16, 8, 8), (16, 8, 16), (16, 8, 32), (16, 8, 64), (16, 8, 128), (16, 8, 256), (16, 16, 8), (16, 16, 16), (16, 16, 32), (16, 16, 64), (16, 16, 128), (16, 16, 256), (16, 32, 8), (16, 32, 16), (16, 32, 32), (16, 32, 64), (16, 32, 128), (16, 32, 256), (16, 64, 8), (16, 64, 16), (16, 64, 32), (16, 64, 64), (16, 64, 128), (16, 64, 256), (16, 128, 8), (16, 128, 16), (16, 128, 32), (16, 128, 64), (16, 128, 128), (16, 128, 256), (16, 256, 8), (16, 256, 16), (16, 256, 32), (16, 256, 64), (16, 256, 128), (16, 256, 256), (32

# *SVM*

In [8]:
results = []
start = time.time()
for bin in bins:
    for img_size in sizes:
        svm = SVC()

        train_data_ = [preprocess2(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 
            # 'gamma': [0.1, 0.5, 1, 5, 10] 
            'gamma' : ["scale", "auto"],
        }       

        # GridSearchCV
        grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')

        # Huấn luyện
        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

NameError: name 'bin_combinations' is not defined

In [15]:
results = []
start = time.time()

for bin_h, bin_s, bin_v in tqdm(bin_combinations, desc="Processing bin combinations"):
    for img_size in tqdm(sizes, leave=False, desc=f"Processing image size {img_size}"):
        svm = SVC()

        train_data_ = [preprocess_hsv(x, img_size, bin_h, bin_s, bin_v) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 
            # 'gamma': [0.1, 0.5, 1, 5, 10] 
            'gamma' : ["scale", "auto"],
        }       

        grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')

        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": (bin_h, bin_s, bin_v),
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

Processing bin combinations: 100%|██████████| 216/216 [2:06:19<00:00, 35.09s/it]  

DONE
Eplased time: 7579.521668672562


In [16]:
best_model = None
best_score = -float('inf')  # Đặt giá trị thấp nhất ban đầu để so sánh

# Lặp qua tất cả các kết quả "trong results
for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


# In kết quả
print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)

# save parameters
params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

with open('./checkpoint/svm_params_hsv.json', 'w') as f:
    json.dump(params, f)

#save model
joblib.dump(best_model, './checkpoint/svm_best_model_hsv.pkl')


Best Overall Model:
img_size: 128
bin: (180, 8, 8)
Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}
{'img_size': 128, 'bin': (180, 8, 8), 'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}


['./checkpoint/svm_best_model_hsv.pkl']

# *KNN*

In [ ]:
results = []
start = time.time()
for bin in bins:
    for img_size in sizes:
        knn = KNeighborsClassifier()
        train_data_ = [preprocess2(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'n_neighbors': [1, 3, 5, 7, 9],         
            'weights': ['uniform', 'distance'],
            'metric': [
                'euclidean', 'manhattan', 'minkowski', 'chebyshev', 'cosine', 'hamming'
            ]
        }

        # GridSearchCV
        grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')

        # Huấn luyện
        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

DONE
Eplased time: 264.8366196155548


In [10]:
best_model = None
best_score = -float('inf')

for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)

params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

# #save params
# with open('./checkpoint/knn_params.json', 'w') as f:
#     json.dump(params, f)

# # save model 
# joblib.dump(best_model, './checkpoint/knn_best_model.pkl')


Best Overall Model:
img_size: 250
bin: 16
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}
{'img_size': 250, 'bin': 16, 'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


# *Random Forest*

In [7]:
import time
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
results = []

# Start timer
start_time = time.time()
bins = [8, 16, 32, 64, 128, 256]
sizes = [32, 64, 128, 250]  # default image_size = 250x250 

for bin in tqdm(bins, desc="Bins"):
    for img_size in tqdm(sizes, desc="Image Sizes", leave=False):
        rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Use all CPU cores

        train_data_ = [preprocess2(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }

        # GridSearchCV
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

        # Train
        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        })

# End timer
end_time = time.time()
elapsed_time = end_time - start_time

print("DONE")
print(f"Total Time Taken: {elapsed_time:.2f} seconds")

Bins:   0%|          | 0/6 [00:00<?, ?it/s]

Bins: 100%|██████████| 6/6 [11:17<00:00, 112.86s/it]

DONE
Total Time Taken: 677.16 seconds


In [8]:
best_model = None
best_score = -float('inf')

for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)

params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

# #save params
# with open('./checkpoint/rf_params.json', 'w') as f:
#     json.dump(params, f)

# # save model 
# joblib.dump(best_model, './checkpoint/rf_best_model.pkl')


Best Overall Model:
img_size: 128
bin: 8
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
{'img_size': 128, 'bin': 8, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
